# imports

In [1]:
from os import listdir,mkdir
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
import seaborn as sns
import random
from scipy.stats import pearsonr
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from ShaniBA.GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import common_processing_feature_phenotype_matrices, editSampleNames,process_sample_matrix,compute_distance_matrix_general
from SampleFileFunctions import *

from tunneltoamazondb import getengine
from pandas import concat, read_csv, Series, read_sql


MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
done


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'14052018'

# plan:

In [3]:
# GENERATE LIST OF ALL SAMPLE PAIRS FOR pnp515
# merge with relationships
# score pairs accrding to similarity groups
# save

# table generation 

## load PNP515 sample list, and PNP515 relation file:

In [4]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP515','rb') as fp:
    PNP515=pickle.load(fp)
print len(PNP515)

515


In [5]:
file1='%s/Sample files/PNP515-relationships.xlsx' %MyPath
PNP515rel=pd.read_excel(file1)
PNP515rel.head()

,BD,Relations,Relative,is relative in PNP515?,comments,RelGroup,SameGroup,BloodGroup
0,BD12,husband of BD121(568087),BD121,1,NaN,2.0,NaN,NaN
1,BD121,wife of BD12(388380),BD12,1,NaN,2.0,NaN,NaN
2,BD124,husband of BD127(225255),BD127,1,NaN,3.0,NaN,NaN
3,BD127,wife of BD124(345928),BD124,1,NaN,3.0,NaN,NaN
4,BD146,wife of BD881(595162),BD881,1,NaN,4.0,NaN,NaN


In [6]:
PNP515rel_toUse=PNP515rel[['BD','Relations','RelGroup','SameGroup','BloodGroup']]

## generate sample pair list:



In [7]:
sample1list=[]
sample2list=[]

for i in range(len(PNP515)):
    for j in range(i+1,len(PNP515)):
        sample1=PNP515[i]
        sample2=PNP515[j]
        sample1list.append(sample1)
        sample2list.append(sample2)
        
PNP515samplePairScoring=pd.DataFrame({'sample1':sample1list,'sample2':sample2list})
print len(PNP515samplePairScoring)
print 515*515/2-515/2
PNP515samplePairScoring.head()

132355
132355


,sample1,sample2
0,BD438,BD690
1,BD438,BD912
2,BD438,BD8
3,BD438,BD101
4,BD438,BD836


In [8]:
# df['new_col'] = list(zip(df.lat, df.long))


PNP515samplePairScoring['samplePair']=list([tuple(sorted(l)) for l in zip(PNP515samplePairScoring['sample1'],PNP515samplePairScoring['sample2'])])

In [9]:
PNP515samplePairScoring.head()

,sample1,sample2,samplePair
0,BD438,BD690,"(BD438, BD690)"
1,BD438,BD912,"(BD438, BD912)"
2,BD438,BD8,"(BD438, BD8)"
3,BD438,BD101,"(BD101, BD438)"
4,BD438,BD836,"(BD438, BD836)"


## add relation information:

In [10]:
PNP515samplePairScoring1=pd.merge(PNP515samplePairScoring,PNP515rel_toUse,how='left',left_on='sample1',right_on='BD')
PNP515samplePairScoring1=PNP515samplePairScoring1.drop('BD',axis=1)
for column in PNP515rel_toUse.columns.values:
    PNP515samplePairScoring1=PNP515samplePairScoring1.rename(columns={column:column+'_sample1'})
    
    
PNP515samplePairScoring1=pd.merge(PNP515samplePairScoring1,PNP515rel_toUse,how='left',left_on='sample2',right_on='BD')
PNP515samplePairScoring1=PNP515samplePairScoring1.drop('BD',axis=1)
for column in PNP515rel_toUse.columns.values:
    PNP515samplePairScoring1=PNP515samplePairScoring1.rename(columns={column:column+'_sample2'})

    

PNP515samplePairScoring1.head()    

,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2
0,BD438,BD690,"(BD438, BD690)",NaN,NaN,NaN,NaN,"mother of BD5(462009),don’t live together",30.0,NaN,17.0
1,BD438,BD912,"(BD438, BD912)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BD438,BD8,"(BD438, BD8)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BD438,BD101,"(BD101, BD438)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BD438,BD836,"(BD438, BD836)",NaN,NaN,NaN,NaN,husband of BD785(458933),44.0,NaN,NaN


## scoring:

In [11]:
#score same samples:
PNP515samplePairScoring1['sameScore']=np.where(PNP515samplePairScoring1['SameGroup_sample1']==\
PNP515samplePairScoring1['SameGroup_sample2'],1,0)
print len(PNP515samplePairScoring1[PNP515samplePairScoring1['sameScore']==1])
PNP515samplePairScoring1[PNP515samplePairScoring1['sameScore']==1]

2


,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore
17184,BD838,BD617,"(BD617, BD838)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD838. wife of BD615/BD839(...,35.0,3.0,3.0,1
100008,BD705,BD714,"(BD705, BD714)",same person as BD714,41.0,4.0,4.0,same person as BD705,41.0,4.0,4.0,1


In [12]:
#score blood relations:
PNP515samplePairScoring1['bloodScore']=np.where(PNP515samplePairScoring1['BloodGroup_sample1']==\
PNP515samplePairScoring1['BloodGroup_sample2'],1,0)
print len(PNP515samplePairScoring1[PNP515samplePairScoring1['bloodScore']==1])
PNP515samplePairScoring1[PNP515samplePairScoring1['bloodScore']==1]

17


,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore,bloodScore
989,BD690,BD5,"(BD5, BD690)","mother of BD5(462009),don’t live together",30.0,NaN,17.0,"son of BD690(121123),don’t live together",30.0,NaN,17.0,0,1
3132,BD782,BD460,"(BD460, BD782)","wife of BD781(454668),mother of BD59(781374), ...",27.0,NaN,16.0,"sister of BD59(781374),daughter of BD781(45466...",27.0,NaN,16.0,0,1
8017,BD349,BD352,"(BD349, BD352)","sister of BD352(555992), mother of BD347(95894...",12.0,NaN,11.0,"sister of BD349(923080),don't live together",12.0,NaN,11.0,0,1
17184,BD838,BD617,"(BD617, BD838)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD838. wife of BD615/BD839(...,35.0,3.0,3.0,1,1
17555,BD375,BD388,"(BD375, BD388)","sister of BD388(387568),daughter of BD62(62574...",14.0,NaN,13.0,"brother of BD375(173217),son of BD62(625742),d...",14.0,NaN,13.0,0,1
19323,BD150,BD151,"(BD150, BD151)",father of BD151(259814),6.0,NaN,8.0,daughter of BD150(120404),6.0,NaN,8.0,0,1
40219,BD412,BD428,"(BD412, BD428)","father of BD430(772166),father of BD428(244004...",20.0,NaN,15.0,"son of BD412(897636), brother of BD430(772166)...",20.0,NaN,15.0,0,1
45800,BD503,BD908,"(BD503, BD908)",wife of BD908(622315) mother of BD533,31.0,NaN,18.0,husband of BD503(366052),31.0,NaN,18.0,0,1
45810,BD503,BD533,"(BD503, BD533)",wife of BD908(622315) mother of BD533,31.0,NaN,18.0,son of BD908(622315) and BD503(366052),31.0,NaN,18.0,0,1
59573,BD692,BD670,"(BD670, BD692)",son of BD670(66206) and BD669(166966) don’t li...,39.0,NaN,20.0,"wife of BD669(166966),mother of BD692(686559),...",39.0,NaN,20.0,0,1


In [13]:
#score enviromental relations:
PNP515samplePairScoring1['environmentalScore']=np.where(PNP515samplePairScoring1['RelGroup_sample1']==\
PNP515samplePairScoring1['RelGroup_sample2'],1,0)
print len(PNP515samplePairScoring1[PNP515samplePairScoring1['environmentalScore']==1])
PNP515samplePairScoring1[PNP515samplePairScoring1['environmentalScore']==1]

49


,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore,bloodScore,environmentalScore
989,BD690,BD5,"(BD5, BD690)","mother of BD5(462009),don’t live together",30.0,NaN,17.0,"son of BD690(121123),don’t live together",30.0,NaN,17.0,0,1,1
2598,BD836,BD785,"(BD785, BD836)",husband of BD785(458933),44.0,NaN,NaN,wife of BD836(1),44.0,NaN,NaN,0,0,1
3132,BD782,BD460,"(BD460, BD782)","wife of BD781(454668),mother of BD59(781374), ...",27.0,NaN,16.0,"sister of BD59(781374),daughter of BD781(45466...",27.0,NaN,16.0,0,1,1
8017,BD349,BD352,"(BD349, BD352)","sister of BD352(555992), mother of BD347(95894...",12.0,NaN,11.0,"sister of BD349(923080),don't live together",12.0,NaN,11.0,0,1,1
8260,BD915,BD914,"(BD914, BD915)",wife of BD914(672874),48.0,NaN,NaN,husband of BD915(776220),48.0,NaN,NaN,0,0,1
11768,BD640,BD636,"(BD636, BD640)",husband of BD636(116140),37.0,NaN,NaN,wife of BD640(933054),37.0,NaN,NaN,0,0,1
12464,BD892,BD887,"(BD887, BD892)",husband of BD887(73708),46.0,NaN,NaN,wife of BD892(205949),46.0,NaN,NaN,0,0,1
15310,BD340,BD339,"(BD339, BD340)","wife of BD339(802543),mother of BD306(72),don'...",11.0,NaN,NaN,"husband of BD340(34707),father of BD306(72),do...",11.0,NaN,NaN,0,0,1
17039,BD838,BD839,"(BD838, BD839)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD615. husband of BD617/BD8...,35.0,NaN,NaN,0,0,1
17184,BD838,BD617,"(BD617, BD838)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD838. wife of BD615/BD839(...,35.0,3.0,3.0,1,1,1


In [14]:
#totalScore:

PNP515samplePairScoring1['TotalScore']=PNP515samplePairScoring1['sameScore']+PNP515samplePairScoring1['bloodScore']+\
PNP515samplePairScoring1['environmentalScore']
print len(PNP515samplePairScoring1[PNP515samplePairScoring1['TotalScore']>0])
PNP515samplePairScoring1[PNP515samplePairScoring1['TotalScore']>0]



49


,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore,bloodScore,environmentalScore,TotalScore
989,BD690,BD5,"(BD5, BD690)","mother of BD5(462009),don’t live together",30.0,NaN,17.0,"son of BD690(121123),don’t live together",30.0,NaN,17.0,0,1,1,2
2598,BD836,BD785,"(BD785, BD836)",husband of BD785(458933),44.0,NaN,NaN,wife of BD836(1),44.0,NaN,NaN,0,0,1,1
3132,BD782,BD460,"(BD460, BD782)","wife of BD781(454668),mother of BD59(781374), ...",27.0,NaN,16.0,"sister of BD59(781374),daughter of BD781(45466...",27.0,NaN,16.0,0,1,1,2
8017,BD349,BD352,"(BD349, BD352)","sister of BD352(555992), mother of BD347(95894...",12.0,NaN,11.0,"sister of BD349(923080),don't live together",12.0,NaN,11.0,0,1,1,2
8260,BD915,BD914,"(BD914, BD915)",wife of BD914(672874),48.0,NaN,NaN,husband of BD915(776220),48.0,NaN,NaN,0,0,1,1
11768,BD640,BD636,"(BD636, BD640)",husband of BD636(116140),37.0,NaN,NaN,wife of BD640(933054),37.0,NaN,NaN,0,0,1,1
12464,BD892,BD887,"(BD887, BD892)",husband of BD887(73708),46.0,NaN,NaN,wife of BD892(205949),46.0,NaN,NaN,0,0,1,1
15310,BD340,BD339,"(BD339, BD340)","wife of BD339(802543),mother of BD306(72),don'...",11.0,NaN,NaN,"husband of BD340(34707),father of BD306(72),do...",11.0,NaN,NaN,0,0,1,1
17039,BD838,BD839,"(BD838, BD839)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD615. husband of BD617/BD8...,35.0,NaN,NaN,0,0,1,1
17184,BD838,BD617,"(BD617, BD838)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD838. wife of BD615/BD839(...,35.0,3.0,3.0,1,1,1,3


In [15]:
PNP515samplePairScoring1.sort_values(by='TotalScore',ascending=False)

,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore,bloodScore,environmentalScore,TotalScore
100008,BD705,BD714,"(BD705, BD714)",same person as BD714,41.0,4.0,4.0,same person as BD705,41.0,4.0,4.0,1,1,1,3
17184,BD838,BD617,"(BD617, BD838)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD838. wife of BD615/BD839(...,35.0,3.0,3.0,1,1,1,3
45800,BD503,BD908,"(BD503, BD908)",wife of BD908(622315) mother of BD533,31.0,NaN,18.0,husband of BD503(366052),31.0,NaN,18.0,0,1,1,2
59573,BD692,BD670,"(BD670, BD692)",son of BD670(66206) and BD669(166966) don’t li...,39.0,NaN,20.0,"wife of BD669(166966),mother of BD692(686559),...",39.0,NaN,20.0,0,1,1,2
17555,BD375,BD388,"(BD375, BD388)","sister of BD388(387568),daughter of BD62(62574...",14.0,NaN,13.0,"brother of BD375(173217),son of BD62(625742),d...",14.0,NaN,13.0,0,1,1,2
105175,BD354,BD919,"(BD354, BD919)","brother of BD919(373276),don't live together",13.0,NaN,12.0,"sister of BD354(291361),don't live together",13.0,NaN,12.0,0,1,1,2
3132,BD782,BD460,"(BD460, BD782)","wife of BD781(454668),mother of BD59(781374), ...",27.0,NaN,16.0,"sister of BD59(781374),daughter of BD781(45466...",27.0,NaN,16.0,0,1,1,2
989,BD690,BD5,"(BD5, BD690)","mother of BD5(462009),don’t live together",30.0,NaN,17.0,"son of BD690(121123),don’t live together",30.0,NaN,17.0,0,1,1,2
19323,BD150,BD151,"(BD150, BD151)",father of BD151(259814),6.0,NaN,8.0,daughter of BD150(120404),6.0,NaN,8.0,0,1,1,2
79819,BD792,BD546,"(BD546, BD792)","brother of BD546(392513),don't live together",32.0,NaN,19.0,"brother of BD792(608673),don't live together",32.0,NaN,19.0,0,1,1,2


## add color columns:

In [16]:
scored=PNP515samplePairScoring1[PNP515samplePairScoring1['TotalScore']>0]

In [17]:
#RelGrop color:
colorList=[]
for n in scored.index:
    if scored.loc[n,'RelGroup_sample1']==scored.loc[n,'RelGroup_sample2']:
        score=int(scored.loc[n,'RelGroup_sample1'])
        color=plt.cm.get_cmap('gist_rainbow')(score*5)
        colorList.append(color)
    else:
        colorList.append(0)
scored['RelGroupColor']=colorList
scored

/usr/wisdom/python/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore,bloodScore,environmentalScore,TotalScore,RelGroupColor
989,BD690,BD5,"(BD5, BD690)","mother of BD5(462009),don’t live together",30.0,NaN,17.0,"son of BD690(121123),don’t live together",30.0,NaN,17.0,0,1,1,2,"(0.0, 0.9878516624040916, 1.0, 1.0)"
2598,BD836,BD785,"(BD785, BD836)",husband of BD785(458933),44.0,NaN,NaN,wife of BD836(1),44.0,NaN,NaN,0,0,1,1,"(0.5040494458653029, 0.0, 1.0, 1.0)"
3132,BD782,BD460,"(BD460, BD782)","wife of BD781(454668),mother of BD59(781374), ...",27.0,NaN,16.0,"sister of BD59(781374),daughter of BD781(45466...",27.0,NaN,16.0,0,1,1,2,"(0.0, 1.0, 0.6957621758380775, 1.0)"
8017,BD349,BD352,"(BD349, BD352)","sister of BD352(555992), mother of BD347(95894...",12.0,NaN,11.0,"sister of BD349(923080),don't live together",12.0,NaN,11.0,0,1,1,2,"(0.8903020667726549, 1.0, 0.0, 1.0)"
8260,BD915,BD914,"(BD914, BD915)",wife of BD914(672874),48.0,NaN,NaN,husband of BD915(776220),48.0,NaN,NaN,0,0,1,1,"(0.9303069053708444, 0.0, 1.0, 1.0)"
11768,BD640,BD636,"(BD636, BD640)",husband of BD636(116140),37.0,NaN,NaN,wife of BD640(933054),37.0,NaN,NaN,0,0,1,1,"(0.0, 0.24190110826939448, 1.0, 1.0)"
12464,BD892,BD887,"(BD887, BD892)",husband of BD887(73708),46.0,NaN,NaN,wife of BD892(205949),46.0,NaN,NaN,0,0,1,1,"(0.7171781756180736, 0.0, 1.0, 1.0)"
15310,BD340,BD339,"(BD339, BD340)","wife of BD339(802543),mother of BD306(72),don'...",11.0,NaN,NaN,"husband of BD340(34707),father of BD306(72),do...",11.0,NaN,NaN,0,0,1,1,"(0.9962904080551138, 1.0, 0.0, 1.0)"
17039,BD838,BD839,"(BD838, BD839)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD615. husband of BD617/BD8...,35.0,NaN,NaN,0,0,1,1,"(0.0, 0.4550298380221651, 1.0, 1.0)"
17184,BD838,BD617,"(BD617, BD838)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD838. wife of BD615/BD839(...,35.0,3.0,3.0,1,1,1,3,"(0.0, 0.4550298380221651, 1.0, 1.0)"


In [18]:
#bloodGrop color:
colorList=[]
for n in scored.index:
    if scored.loc[n,'BloodGroup_sample1']==scored.loc[n,'BloodGroup_sample2']:
        score=int(scored.loc[n,'BloodGroup_sample1'])
        color=plt.cm.get_cmap('gist_rainbow')(score*10)
        colorList.append(color)
    else:
        colorList.append((0.0,0.0,0.0,0.0))
scored['BloodGroupColor']=colorList
scored

/usr/wisdom/python/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore,bloodScore,environmentalScore,TotalScore,RelGroupColor,BloodGroupColor
989,BD690,BD5,"(BD5, BD690)","mother of BD5(462009),don’t live together",30.0,NaN,17.0,"son of BD690(121123),don’t live together",30.0,NaN,17.0,0,1,1,2,"(0.0, 0.9878516624040916, 1.0, 1.0)","(0.0, 0.5615942028985503, 1.0, 1.0)"
2598,BD836,BD785,"(BD785, BD836)",husband of BD785(458933),44.0,NaN,NaN,wife of BD836(1),44.0,NaN,NaN,0,0,1,1,"(0.5040494458653029, 0.0, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)"
3132,BD782,BD460,"(BD460, BD782)","wife of BD781(454668),mother of BD59(781374), ...",27.0,NaN,16.0,"sister of BD59(781374),daughter of BD781(45466...",27.0,NaN,16.0,0,1,1,2,"(0.0, 1.0, 0.6957621758380775, 1.0)","(0.0, 0.774722932651321, 1.0, 1.0)"
8017,BD349,BD352,"(BD349, BD352)","sister of BD352(555992), mother of BD347(95894...",12.0,NaN,11.0,"sister of BD349(923080),don't live together",12.0,NaN,11.0,0,1,1,2,"(0.8903020667726549, 1.0, 0.0, 1.0)","(0.0, 1.0, 0.16866961838498848, 1.0)"
8260,BD915,BD914,"(BD914, BD915)",wife of BD914(672874),48.0,NaN,NaN,husband of BD915(776220),48.0,NaN,NaN,0,0,1,1,"(0.9303069053708444, 0.0, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)"
11768,BD640,BD636,"(BD636, BD640)",husband of BD636(116140),37.0,NaN,NaN,wife of BD640(933054),37.0,NaN,NaN,0,0,1,1,"(0.0, 0.24190110826939448, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)"
12464,BD892,BD887,"(BD887, BD892)",husband of BD887(73708),46.0,NaN,NaN,wife of BD892(205949),46.0,NaN,NaN,0,0,1,1,"(0.7171781756180736, 0.0, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)"
15310,BD340,BD339,"(BD339, BD340)","wife of BD339(802543),mother of BD306(72),don'...",11.0,NaN,NaN,"husband of BD340(34707),father of BD306(72),do...",11.0,NaN,NaN,0,0,1,1,"(0.9962904080551138, 1.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0)"
17039,BD838,BD839,"(BD838, BD839)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD615. husband of BD617/BD8...,35.0,NaN,NaN,0,0,1,1,"(0.0, 0.4550298380221651, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)"
17184,BD838,BD617,"(BD617, BD838)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD838. wife of BD615/BD839(...,35.0,3.0,3.0,1,1,1,3,"(0.0, 0.4550298380221651, 1.0, 1.0)","(1.0, 0.47376788553259147, 0.0, 1.0)"


In [19]:
#sameGrop color:
colorList=[]
for n in scored.index:
    if scored.loc[n,'SameGroup_sample1']==scored.loc[n,'SameGroup_sample2']:
        score=int(scored.loc[n,'SameGroup_sample1'])
        color=plt.cm.get_cmap('gist_rainbow')(score*20)
        colorList.append(color)
    else:
        colorList.append(0)
scored['SameGroupColor']=colorList
scored

/usr/wisdom/python/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore,bloodScore,environmentalScore,TotalScore,RelGroupColor,BloodGroupColor,SameGroupColor
989,BD690,BD5,"(BD5, BD690)","mother of BD5(462009),don’t live together",30.0,NaN,17.0,"son of BD690(121123),don’t live together",30.0,NaN,17.0,0,1,1,2,"(0.0, 0.9878516624040916, 1.0, 1.0)","(0.0, 0.5615942028985503, 1.0, 1.0)",0
2598,BD836,BD785,"(BD785, BD836)",husband of BD785(458933),44.0,NaN,NaN,wife of BD836(1),44.0,NaN,NaN,0,0,1,1,"(0.5040494458653029, 0.0, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0
3132,BD782,BD460,"(BD460, BD782)","wife of BD781(454668),mother of BD59(781374), ...",27.0,NaN,16.0,"sister of BD59(781374),daughter of BD781(45466...",27.0,NaN,16.0,0,1,1,2,"(0.0, 1.0, 0.6957621758380775, 1.0)","(0.0, 0.774722932651321, 1.0, 1.0)",0
8017,BD349,BD352,"(BD349, BD352)","sister of BD352(555992), mother of BD347(95894...",12.0,NaN,11.0,"sister of BD349(923080),don't live together",12.0,NaN,11.0,0,1,1,2,"(0.8903020667726549, 1.0, 0.0, 1.0)","(0.0, 1.0, 0.16866961838498848, 1.0)",0
8260,BD915,BD914,"(BD914, BD915)",wife of BD914(672874),48.0,NaN,NaN,husband of BD915(776220),48.0,NaN,NaN,0,0,1,1,"(0.9303069053708444, 0.0, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0
11768,BD640,BD636,"(BD636, BD640)",husband of BD636(116140),37.0,NaN,NaN,wife of BD640(933054),37.0,NaN,NaN,0,0,1,1,"(0.0, 0.24190110826939448, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0
12464,BD892,BD887,"(BD887, BD892)",husband of BD887(73708),46.0,NaN,NaN,wife of BD892(205949),46.0,NaN,NaN,0,0,1,1,"(0.7171781756180736, 0.0, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0
15310,BD340,BD339,"(BD339, BD340)","wife of BD339(802543),mother of BD306(72),don'...",11.0,NaN,NaN,"husband of BD340(34707),father of BD306(72),do...",11.0,NaN,NaN,0,0,1,1,"(0.9962904080551138, 1.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0
17039,BD838,BD839,"(BD838, BD839)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD615. husband of BD617/BD8...,35.0,NaN,NaN,0,0,1,1,"(0.0, 0.4550298380221651, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0
17184,BD838,BD617,"(BD617, BD838)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD838. wife of BD615/BD839(...,35.0,3.0,3.0,1,1,1,3,"(0.0, 0.4550298380221651, 1.0, 1.0)","(1.0, 0.47376788553259147, 0.0, 1.0)","(0.8903020667726549, 1.0, 0.0, 1.0)"


In [20]:
#totalScore color:
colorList=[]
for n in scored.index:
    score=int(scored.loc[n,'TotalScore'])
    if score==0:
        color=plt.cm.get_cmap('Set1')(9)
    else:
        color=plt.cm.get_cmap('gist_rainbow')(score*20)
    colorList.append(color)
scored['TotalScoreColor']=colorList
scored

/usr/wisdom/python/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore,bloodScore,environmentalScore,TotalScore,RelGroupColor,BloodGroupColor,SameGroupColor,TotalScoreColor
989,BD690,BD5,"(BD5, BD690)","mother of BD5(462009),don’t live together",30.0,NaN,17.0,"son of BD690(121123),don’t live together",30.0,NaN,17.0,0,1,1,2,"(0.0, 0.9878516624040916, 1.0, 1.0)","(0.0, 0.5615942028985503, 1.0, 1.0)",0,"(1.0, 0.6857445680975094, 0.0, 1.0)"
2598,BD836,BD785,"(BD785, BD836)",husband of BD785(458933),44.0,NaN,NaN,wife of BD836(1),44.0,NaN,NaN,0,0,1,1,"(0.5040494458653029, 0.0, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0,"(1.0, 0.2617912029676736, 0.0, 1.0)"
3132,BD782,BD460,"(BD460, BD782)","wife of BD781(454668),mother of BD59(781374), ...",27.0,NaN,16.0,"sister of BD59(781374),daughter of BD781(45466...",27.0,NaN,16.0,0,1,1,2,"(0.0, 1.0, 0.6957621758380775, 1.0)","(0.0, 0.774722932651321, 1.0, 1.0)",0,"(1.0, 0.6857445680975094, 0.0, 1.0)"
8017,BD349,BD352,"(BD349, BD352)","sister of BD352(555992), mother of BD347(95894...",12.0,NaN,11.0,"sister of BD349(923080),don't live together",12.0,NaN,11.0,0,1,1,2,"(0.8903020667726549, 1.0, 0.0, 1.0)","(0.0, 1.0, 0.16866961838498848, 1.0)",0,"(1.0, 0.6857445680975094, 0.0, 1.0)"
8260,BD915,BD914,"(BD914, BD915)",wife of BD914(672874),48.0,NaN,NaN,husband of BD915(776220),48.0,NaN,NaN,0,0,1,1,"(0.9303069053708444, 0.0, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0,"(1.0, 0.2617912029676736, 0.0, 1.0)"
11768,BD640,BD636,"(BD636, BD640)",husband of BD636(116140),37.0,NaN,NaN,wife of BD640(933054),37.0,NaN,NaN,0,0,1,1,"(0.0, 0.24190110826939448, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0,"(1.0, 0.2617912029676736, 0.0, 1.0)"
12464,BD892,BD887,"(BD887, BD892)",husband of BD887(73708),46.0,NaN,NaN,wife of BD892(205949),46.0,NaN,NaN,0,0,1,1,"(0.7171781756180736, 0.0, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0,"(1.0, 0.2617912029676736, 0.0, 1.0)"
15310,BD340,BD339,"(BD339, BD340)","wife of BD339(802543),mother of BD306(72),don'...",11.0,NaN,NaN,"husband of BD340(34707),father of BD306(72),do...",11.0,NaN,NaN,0,0,1,1,"(0.9962904080551138, 1.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0,"(1.0, 0.2617912029676736, 0.0, 1.0)"
17039,BD838,BD839,"(BD838, BD839)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD615. husband of BD617/BD8...,35.0,NaN,NaN,0,0,1,1,"(0.0, 0.4550298380221651, 1.0, 1.0)","(0.0, 0.0, 0.0, 0.0)",0,"(1.0, 0.2617912029676736, 0.0, 1.0)"
17184,BD838,BD617,"(BD617, BD838)",The same person as BD617. wife of BD615/BD839(...,35.0,3.0,3.0,The same person as BD838. wife of BD615/BD839(...,35.0,3.0,3.0,1,1,1,3,"(0.0, 0.4550298380221651, 1.0, 1.0)","(1.0, 0.47376788553259147, 0.0, 1.0)","(0.8903020667726549, 1.0, 0.0, 1.0)","(0.8903020667726549, 1.0, 0.0, 1.0)"


In [21]:
scored_toUse=scored[['sample1','sample2','RelGroupColor','BloodGroupColor','SameGroupColor','TotalScoreColor']]
PNP515samplePairScoring2=pd.merge(PNP515samplePairScoring1,scored_toUse,how='left',left_on=['sample1','sample2'], right_on=['sample1','sample2'])
print len(PNP515samplePairScoring1)
print len(PNP515samplePairScoring2)

132355
132355


In [22]:
PNP515samplePairScoring2.head()

,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore,bloodScore,environmentalScore,TotalScore,RelGroupColor,BloodGroupColor,SameGroupColor,TotalScoreColor
0,BD438,BD690,"(BD438, BD690)",NaN,NaN,NaN,NaN,"mother of BD5(462009),don’t live together",30.0,NaN,17.0,0,0,0,0,NaN,NaN,NaN,NaN
1,BD438,BD912,"(BD438, BD912)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN
2,BD438,BD8,"(BD438, BD8)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN
3,BD438,BD101,"(BD101, BD438)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN
4,BD438,BD836,"(BD438, BD836)",NaN,NaN,NaN,NaN,husband of BD785(458933),44.0,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN


In [23]:
for column in ['RelGroupColor','BloodGroupColor','SameGroupColor','TotalScoreColor']:
    PNP515samplePairScoring2[column]=PNP515samplePairScoring2[column].fillna(0)
PNP515samplePairScoring2.head()

,sample1,sample2,samplePair,Relations_sample1,RelGroup_sample1,SameGroup_sample1,BloodGroup_sample1,Relations_sample2,RelGroup_sample2,SameGroup_sample2,BloodGroup_sample2,sameScore,bloodScore,environmentalScore,TotalScore,RelGroupColor,BloodGroupColor,SameGroupColor,TotalScoreColor
0,BD438,BD690,"(BD438, BD690)",NaN,NaN,NaN,NaN,"mother of BD5(462009),don’t live together",30.0,NaN,17.0,0,0,0,0,0,0,0,0
1,BD438,BD912,"(BD438, BD912)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
2,BD438,BD8,"(BD438, BD8)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
3,BD438,BD101,"(BD101, BD438)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
4,BD438,BD836,"(BD438, BD836)",NaN,NaN,NaN,NaN,husband of BD785(458933),44.0,NaN,NaN,0,0,0,0,0,0,0,0


## save:

In [24]:
file2='%s/Sample files/PNP515-relationshipsScoring' %MyPath
PNP515samplePairScoring2.to_pickle(file2)

In [25]:
file1='%s/Sample files/PNP515-relationshipsScoring.xlsx' %MyPath
PNP515samplePairScoring2.to_excel(file1)


In [26]:
print 'dfdfd'

dfdfd
